In [1]:
import pyspark
sc = pyspark.SparkContext()

In [56]:
import sqlite3
import datetime
import time
from time import strptime
from datetime import datetime

from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import RandomForest


bd = sqlite3.connect('velib3.db')
c = bd.cursor()

def feedData(jour, mois, annee, num_station):

    #select * from station_dyn where num = 10114  and db_write_time >= ('16/03/2017') and db_write_time < ('17/03/2017') order by last_update

    enregistrements=[]

    week_day = datetime(annee, mois, jour).timetuple()[6]
    if week_day not in (5,6):
        hist=[-1,-2,-3,-7,-14]
    else:
        hist=[-7,-8,-14,-15]
        if week_day == 6:
            hist.append(-1)

    for i in hist:
        if datetime(annee, mois, jour+i).timetuple()[6] in (5,6):
            hist.append(i-2)
            continue
        clause ="last_update_date >= (\'"+str(datetime(annee, mois, jour+i))[:10]+"\') and last_update_date < (\'"+ str(datetime(annee, mois, jour+i+1))[:10]+"\')"
        c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where "+clause+" and num = "+str(num_station)+" order by last_update")
        enregistrements.extend(c.fetchall())
        enregistrements=sorted(enregistrements, key=lambda e: e[1])
    #print enregistrements
    #print len(enregistrements)
    
    return enregistrements

#test = feedData(28, 3, 2017, 15003)
#print test

def predictData(jour, mois, annee, num_station):
    enregistrements=[]
    clause ="last_update_date >= (\'"+str(datetime(annee, mois, jour))[:10]+"\') and last_update_date < (\'"+ str(datetime(annee, mois, jour+1))[:10]+"\')"
    c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where "+clause+" and num = "+str(num_station)+" order by last_update")
    enregistrements.extend(c.fetchall())
    enregistrements=sorted(enregistrements, key=lambda e: e[1])
    #print enregistrements
    #print len(enregistrements)
    
    return enregistrements

#test = predictData(28, 3, 2017, 15003)
#print test

c.execute("select DISTINCT num from station_dyn" )

list_station=[]

for row in c:
    list_station.append(row[0]) 

"""
for h in range(0,24):
    for m in range(0,51,10):
        #print ("0" if len(str(h))==1 else "")+str(h)+":"+str(m)+("0" if m==0 else "") #144 intervalles par jour
        pass"""

data=[]
interval_pred=600*3

fichier = open("data.txt", "w")
#fichier = open("data_2.txt", "w")





for station in range(15003,15004):#0,1):#len(list_station)):
    list_row=feedData(28, 3, 2017, 15003)
    #list_row=predictData(28, 3, 2017, 15003)
    
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(15003)+" order by last_update")
    #c.execute("select nb_velo_dispo, last_update, last_update_date from station_dyn where num = "+str(list_station[station])+" order by last_update")
    
    #list_row=c.fetchall()
    
    """i=0
    while list_row!=[]:
        print list_row.pop(0)
        i+=1
        print str(i)+(" vide" if list_row==[] else " non vide")
    print len(list_row)
    break"""
    
    #print list_row[-1]
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[-1][1])/1000/600)*600))
    #print time.strftime("%d/%m/%Y-%H:%M:%S", time.gmtime((int(list_row[0][1])/1000/600)*600+600))
    
    row=1
    boucle=0
    in_flow=0
    out_flow=0
    date_interval = (int(list_row[0][1])/1000/interval_pred)*interval_pred+interval_pred
    #for date in range((int(list_row[0][1])/1000/600)*600+600,(int(list_row[-1][1])/1000/600)*600+1,600):
        #print datetime.datetime.fromtimestamp(date).strftime('%H:%M'), datetime.datetime.fromtimestamp(date).strftime('%d-%m')
        
        #implémenter les filtres
    prec_row=list_row[0]
    current_row=list_row.pop(0) #Erreur si la liste est vide
    while list_row!=[]:
        current_row=list_row.pop(0)
        row+=1
        
        flow=current_row[0]-prec_row[0]
        
        if int(current_row[1])/1000<date_interval:
            boucle+=1
            in_flow += flow if flow>0 else 0
            out_flow += flow if flow<0 else 0
        else: #in_flow : nb_velo_dispo, week-day, heure_normalisée
            #data.append(LabeledPoint(float(in_flow),[float(current_row[0]),time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30]))
            #=>data.append(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000)[6],int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30]))

            data.append(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))[6],int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30]))
            
            #in_flow 1:heure_normalisée 2:week-day
            retour_chariot = '\n' if row!=3 else ""
            fichier.write(retour_chariot+str(float(in_flow))+" 1:"+str(int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30)+" 2:"+str(time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))[6]))
            #fichier.write('\n'+str(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))[6],int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30])))

            
            #data.append(LabeledPoint(float(in_flow),[float((time.gmtime(int(current_row[1])/1000)[6]))/6,float((int(time.strftime("%H", time.gmtime(date_interval+3600-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+3600-interval_pred)))/30))/47]))

            print [LabeledPoint(float(in_flow),\
                   [time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)[6],\
                   int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30])]
            while int(current_row[1])/1000>=date_interval:
                date_interval+=interval_pred
            boucle=1
            in_flow = flow if flow>0 else 0
            out_flow = flow if flow<0 else 0
        
        print boucle, row, current_row, time.strftime("%H:%M:%S", time.gmtime((int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))-(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))
        print "intervalle : ["+str(time.strftime("%H:%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))+","+str(time.strftime("%H:%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200))))+"["+" -  n° : "+str(int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30) #doit être modifié à la main si changemenr d'intervalle
        
        print "in flow : "+str(in_flow)+" - out flow : "+str(out_flow)
        
        prec_row=current_row
    
    data.append(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))[6],int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30]))
    retour_chariot = '\n' if row!=3 else ""
    fichier.write(retour_chariot+str(float(in_flow))+" 1:"+str(int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30)+" 2:"+str(time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)[6]))
    #fichier.write('\n'+str(LabeledPoint(float(in_flow),[time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200))[6],int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30])))
    print [LabeledPoint(float(in_flow),\
    [time.gmtime(int(current_row[1])/1000+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)[6],\
    int(time.strftime("%H", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))*2+int(time.strftime("%M", time.gmtime(date_interval+(3600 if int(current_row[1]) < 1490490000000 else 7200)-interval_pred)))/30])]
            
    fichier.close()

c.close()

1 2 (0, u'1490653076000', u'2017-03-28 00:17:56') 00:17:56
intervalle : [00:00,00:30[ -  n° : 0
in flow : 0 - out flow : 0
2 3 (0, u'1490653680000', u'2017-03-28 00:28:00') 00:28:00
intervalle : [00:00,00:30[ -  n° : 0
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [1.0,0.0])]
1 4 (0, u'1490654284000', u'2017-03-28 00:38:04') 00:08:04
intervalle : [00:30,01:00[ -  n° : 1
in flow : 0 - out flow : 0
2 5 (0, u'1490654887000', u'2017-03-28 00:48:07') 00:18:07
intervalle : [00:30,01:00[ -  n° : 1
in flow : 0 - out flow : 0
3 6 (0, u'1490655491000', u'2017-03-28 00:58:11') 00:28:11
intervalle : [00:30,01:00[ -  n° : 1
in flow : 0 - out flow : 0
[LabeledPoint(0.0, [1.0,1.0])]
1 7 (0, u'1490656095000', u'2017-03-28 01:08:15') 00:08:15
intervalle : [01:00,01:30[ -  n° : 2
in flow : 0 - out flow : 0
2 8 (0, u'1490656699000', u'2017-03-28 01:18:19') 00:18:19
intervalle : [01:00,01:30[ -  n° : 2
in flow : 0 - out flow : 0
3 9 (0, u'1490657302000', u'2017-03-28 01:28:22') 00:28:22
intervalle : [01:0

In [22]:
time.gmtime(int('1490050845000')/1000)[6]

0

In [ ]:
print len(data)
print "********"
print data

In [ ]:
from pyspark.mllib.classification import LogisticRegressionWithLBFGS, LogisticRegressionModel, LogisticRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint

model = LogisticRegressionWithLBFGS.train(sc.parallelize(data), numClasses=17,iterations=5)

print model
pred = model.predict(sc.parallelize(data).map(lambda x:x.features))
#pred.take(10)
LabelsAndPredictions = sc.parallelize(data).map(lambda x:x.label).zip(pred.map(lambda x:float(x)))
LabelsAndPredictions.count()
LabelsAndPredictions.collect()


In [ ]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint

svm = SVMWithSGD.train(sc.parallelize(data), iterations=100)
pred2 = svm.predict(sc.parallelize(data).map(lambda x:x.features))
labelsAndPreds2 = sc.parallelize(data).map(lambda x:x.label).zip(pred2.map(lambda x:float(x)))
labelsAndPreds2.collect()

labelsAndPreds = sc.parallelize(data).map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(data.count())
print("Training Error = " + str(trainErr))

In [ ]:




# Save and load model
model.save(sc, "target/tmp/pythonSVMWithSGDModel")
sameModel = SVMModel.load(sc, "target/tmp/pythonSVMWithSGDModel")

In [ ]:
data = [LabeledPoint(0.0, [0.0]),LabeledPoint(1.0, [1.0]),LabeledPoint(1.0, [2.0]),LabeledPoint(1.0, [3.0])]
svm = SVMWithSGD.train(sc.parallelize(data), iterations=10)
svm.predict([1.0])
svm.predict(sc.parallelize([[1.0]])).collect()